# Hello! Thank you for giving GMT/Python a try.


This is a [Jupyter notebook](http://jupyter.org). It's an interactive computing environment where you can mix text (like this), code, and figures. The notebook is organized into *cells*. This is a Markdown cell (click on it to see the source) and it can contain text, hyperlinks, images, and even Latex equations.

To execute any cell, click on it and type `Shift + Enter` or click on the "Run" button in the menu bar. Executing a Markdown cell will render its content. Code execution can happen non-linearly, so you can change and rerun a cell without running all of the ones that came before it. But you'll still need to run cells that define a variable/import a module before you can use the variable/module in another cell. You can restart and clear the notebook at any time in the "Kernel" menu.

This is an example of what you can currently do with GMT/Python. There are some empty cells at the bottom of this notebook for you to experiment along with an example using the command-line version of GMT6 in [modern mode](http://gmt.soest.hawaii.edu/projects/gmt/wiki/Modernization). 

Feel free to **experiment, change the code, and create new cells**. If you run into any problems or bugs, please [create a Github issue](https://github.com/GenericMappingTools/gmt-python/issues) explaining what went wrong.

## Loading the library

The GMT modules are available as functions and classes in the `gmt` Python package. 
So we'll start by importing it:

In [ ]:
import gmt

## A first map

All figure generation in GMT/Python is handled by the `gmt.Figure` class. 
It has methods to add layers to your figure, like a basemap, coastlines, and data.

We start a new figure by creating an instance of `gmt.Figure`:

In [ ]:
fig = gmt.Figure()

We add elements to the figure using its methods. For example, lets add the coastlines of Central America to a 6 inch wide map using the Mercator projection (`M`). Our figure will also have a nice frame with automatic ticks.

In [ ]:
fig.coast(region=[-90, -70, 0, 20], projection='M6i', land='chocolate', frame=True)

You can see a preview of the figure directly in the notebook using `fig.show()`.

In [ ]:
fig.show()

## A note for experienced GMT users

You'll probably have noticed several things that are different from classic command-line GMT.
Many of these changes reflect the new GMT [modern execution mode](http://gmt.soest.hawaii.edu/projects/gmt/wiki/Modernization) that will be part of the future 6.0 release.
A few are GMT/Python exclusive (like the long argument names).

1. The name of method is `coast` instead of `pscoast`. As a general rule, all `ps*` modules had their `ps` removed. The exceptions are: `psxy == plot`, `psxyz == plot3d`, and `psscale == colorbar`.
2. The arguments don't use the GMT 1-letter syntax (R, J, B, etc). Those are still available as aliases and the methods will accept them (see below). 
3. Arguments like `region` can take lists instead of strings like `1/2/3/4`. You can still use the string form but the list form is easier in Python.
4. If a GMT argument has no options (like `-B` instead of `-Baf`), use a `True` value instead. An empty string would also be acceptable.
5. There is no output redirecting to a PostScript file. The figure is generated in the background and will only be shown or saved when you ask for it.

We could have generated the figure above using the classic GMT argument names (but not the module names):

In [ ]:
fig_alias = gmt.Figure()
fig_alias.coast(R='-90/-70/0/20', J='M6i', G='gray', S='blue', B=True)
fig_alias.show()

## Saving the figure

Unlike the GMT command-line interface, **no figure file was generated until you ask for one**. That means that `fig.show` won't produce a figure file. 

Use method `fig.savefig` (based on the [matplotlib](https://matplotlib.org/) function) if you want to save your figure:

In [ ]:
fig.savefig('central-america.png')

We can check that the file was created using the `ls` shell command:

In [ ]:
%%bash
ls *.png

Another way to view the figure in the notebook is using the `IPython.display.Image` class:

In [ ]:
from IPython.display import Image

In [ ]:
Image('central-america.png', width=400)

## Plotting point data

We can use `gmt.Figure.plot` to plot data on our map. The `plot` method used to be called `psxy` in GMT before version 6.0.

First, lets load some earthquake data (historical quakes from USGS). This dataset is packaged with GMT6 and can be accessed from the command-line by running:

    gmt which @usgs_quakes_22.txt -G
    
The file `usgs_quakes.txt` that's included in this demo is a simplified version of this dataset including only the epicenter coordinates, the magnitude, and the depths of the earthquakes.

In [ ]:
import numpy as np

In [ ]:
lon, lat, magnitude, depth = np.loadtxt('usgs_quakes.txt', unpack=True)

Now we can plot the data using `Figure.plot` and passing the coordinate arrays as the `x` and `y` arguments:

In [ ]:
fig = gmt.Figure()
# Plot using a global region ('g') and an Othographic projection ('G')
fig.coast(region='g', projection='G200/30/6i', frame='g', 
          land='grey', water='white')
# Plot using circles (c) of 0.1 cm
fig.plot(x=lon, y=lat, style='c0.1c', color='black')
fig.show()

We can make the size of the markers follow the magnitude values by passing in the argument `sizes` to `Figure.plot`. We'll need to scale the magnitude (the size is in centimeters) and use a power law to actually see all the different sizes.

In [ ]:
fig = gmt.Figure()
fig.coast(region='g', projection='G200/30/6i', frame='g', 
          land='grey', water='white')
fig.plot(x=lon, y=lat, sizes=0.005*(2**magnitude), style='cc', 
         color='black')
fig.show()

We can also map the colors of the markers to the depths by passing an array to the `color` argument annd providing a colormap name (`cmap`). We can even use the new matplotlib colormap `"viridis"`. In order to highlight the depth range, we'll use the log10 of the depth instead.

In [ ]:
# Only take the log where depth is > 0
log_depth = np.log10(depth, where=depth > 0)
# We need to normalize the color because `plot` cannot autoscale the cmap to our data. This is coming soon to GMT.
log_depth_norm = log_depth/log_depth.max()

We need to normalize the magnitude because currently `plot` cannot autoscale the colormap to your data. This is coming soon to GMT.

In [ ]:
fig = gmt.Figure()
fig.coast(region='g', projection='G200/30/6i', frame='g', 
          land='grey', water='white')
fig.plot(x=lon, y=lat, sizes=0.005*(2**magnitude), style='cc', 
         color=log_depth_norm, cmap='viridis')
fig.show()

Let's plot the same figure as the one above using a Mercator projection and a different colormap.

In [ ]:
fig = gmt.Figure()
fig.coast(region=[-330, 30, -70, 70],  projection='M10i', 
          frame='afg', land='grey', water='white')
fig.plot(x=lon, y=lat, sizes=0.005*(2**magnitude), style='cc', 
         color=log_depth_norm, cmap='ocean')
# Make the preview ocupy the entire browser width
fig.show(width='100%')

# Experiment for yourself

Type anything you want in the cells below. 
Or go back up and change the code in the cells and run them again.
If you run into any problems or bugs, please [create a Github issue](https://github.com/GenericMappingTools/gmt-python/issues) explaining what went wrong.

You can even try the GMT6 command line programs by placing `%%bash` in the top of your cell. Use the `IPython.display.Image` class to embed your figure in the notebook:

In [ ]:
from IPython.display import Image

In [ ]:
%%bash
gmt begin
    gmt figure my-figure png
    gmt grdimage @earth_relief_10m.grd -Cearth -JG200/30/6i -Bafg
    gmt coast -W -Dc -Ggray
    gmt plot @hotspots.txt -Sc0.2c -Gred
gmt end

In [ ]:
Image('my-figure.png', width=500)